Start this first
```bash
$ litex_server uart /dev/ttyUSB0 115200
```

In [1]:
from litex.soc.tools.remote import RemoteClient
from struct import pack, unpack
import time

In [2]:
for i in range(32):
    try:
        r = RemoteClient(csr_csv="build/csr.csv", debug=False, port=1234 + i)
        r.open()
        print("Connected to Port", 1234 + i)
        break
    except ConnectionRefusedError:
        pass

def getId():
    s = ""
    for i in range(64):
        temp = r.read(r.bases.identifier_mem + i * 4)
        if temp == 0:
            break
        s += chr(temp & 0xFF)
    return s
getId()

Connected to Port 1234


'Wir trampeln durchs Getreide ... 2019-04-02 21:31:21'

# SPI config memo


    config signal:
        1 offline: all pins high-z (reset=1)
        1 active: cs/transfer active (read-only)
        1 pending: transfer pending in intermediate buffer (read-only)
        1 cs_polarity: active level of chip select (reset=0)
        1 clk_polarity: idle level of clk (reset=0)
        1 clk_phase: first edge after cs assertion to sample data on (reset=0)
            (clk_polarity, clk_phase) == (CPOL, CPHA) in Freescale language.
            (0, 0): idle low, output on falling, input on rising
            (0, 1): idle low, output on rising, input on falling
            (1, 0): idle high, output on rising, input on falling
            (1, 1): idle high, output on falling, input on rising
            There is never a clk edge during a cs edge.
        1 lsb_first: LSB is the first bit on the wire (reset=0)
        1 half_duplex: 3-wire SPI, in/out on mosi (reset=0)
        8 undefined
        8 div_write: counter load value to divide this module's clock
            to generate the SPI write clk (reset=0)
            f_clk/f_spi_write == div_write + 2
        8 div_read: ditto for the read clock

    xfer_config signal:
        16 cs: active high bit mask of chip selects to assert (reset=0)
        6 write_len: 0-M bits (reset=0)
        2 undefined
        6 read_len: 0-M bits (reset=0)
        2 undefined

# Setup SPI

In [3]:
r.regs.spi_config.write(0xFFFF0000)
# 16 bit write transfer (includes read as is 4 wire)
r.regs.spi_xfer.write(0x0010FFFF)

def set_ltc_reg(adr, val):
    word = (0 << 15) | ((adr & 0x7F) << 8) | (val & 0xFF)
    word <<= 16
    r.regs.spi_mosi_data.write(word)
    r.regs.spi_start.write(1)

def get_ltc_reg(adr):
    word = (1 << 15) | ((adr & 0x7F) << 8)
    word <<= 16
    r.regs.spi_mosi_data.write(word)
    r.regs.spi_start.write(1)
    return r.regs.spi_miso_data.read() & 0xFF

def setTp(tpValue):
    set_ltc_reg(3, (1<<7) | tpValue >> 8)   # Test pattern on + value MSB
    set_ltc_reg(4, tpValue & 0xFF)          # Test pattern value LSB
    
def myzip(*vals):
    return [i for t in zip(*vals) for i in t]    

# Dump LTC2175 registers

In [4]:
for i in range(5):
    print("{:2x}: {:02x}".format(i, get_ltc_reg(i)))

 0: 00
 1: 00
 2: 00
 3: 00
 4: 00


In [680]:
set_ltc_reg(0, 0x80)   # Software reset

In [15]:
set_ltc_reg(1, 0x10)   # Sleep mode

In [16]:
set_ltc_reg(1, 0x00)   # Waky waky

# Measure sample clock frequency [Hz]

In [5]:
r.regs.lvds_f_sample_value.read()

114285714

# Read and align clock data

In [7]:
rVal = r.regs.lvds_clk_peek.read()
print("0x{0:02X} 0b{0:08b}".format(rVal))

0x0F 0b00001111


do a bitslip until we get 0x0F

In [222]:
r.regs.lvds_bitslip_csr.write(1)

# Map data lanes
Check that each bit in the test-pattern end up in the right place in the output word

In [284]:
#print("    Test pattern   LVDS_B   LVDS_A")
for i in range(14):
    tpValue = 1<<i
    setTp(tpValue)
    rVal = r.regs.lvds_data_peek.read()
    print("{:016b} {:016b}".format(tpValue, rVal))
    # rVal_a = rVal & 0xFF
    # rVal_b = (rVal >> 8) & 0xFF
    # print("{:016b} {:08b} {:08b}".format(tpValue, rVal_b, rVal_a))

0000000000000001 0000000000000100
0000000000000010 0000000000001000
0000000000000100 0000000000010000
0000000000001000 0000000000100000
0000000000010000 0000000001000000
0000000000100000 0000000010000000
0000000001000000 0000000100000000
0000000010000000 0000001000000000
0000000100000000 0000010000000000
0000001000000000 0000100000000000
0000010000000000 0001000000000000
0000100000000000 0010000000000000
0001000000000000 0100000000000000
0010000000000000 1000000000000000


# Read ADC sample

In [1061]:
set_ltc_reg(3, 0)   # Test pattern off
set_ltc_reg(1, 0)   # Randomizer off

In [1068]:
# Lowest 2 bits are always zero in 16 bit LVDS mode
# 12 bit ADC, so next 2 bits are also zero
rVal = r.regs.lvds_data_peek.read() >> 4
print("0x{0:03x} 0b{0:012b}".format(rVal))
# Decode offset binary mode
print(rVal - (1 << 11))

0x7fe 0b011111111110
-2


# Read randomized ADC sample

In [967]:
set_ltc_reg(1, (1<<6))   # Randomizer on

In [1060]:
rVal = r.regs.lvds_data_peek.read() >> 4
print("0x{0:03x} 0b{0:012b}".format(rVal))
# XOR LSB with any other bit
if rVal & 0x01:
    rVal ^= 0xFFE
# Decode offset binary mode
print(rVal - (1 << 11))

0x803 0b100000000011
-3


In [42]:
r.close()